In [1]:
#This command needs to be run from the auth environment or it won't run.
#from project import db, create_app
#db.create_all(app=create_app())

In [1]:
import sys, os
sys.path.append('./..')

In [2]:
############################
from werkzeug.security import generate_password_hash, check_password_hash

sys.path.append('/users/taylordupuy/Documents/web-development/dev/excel_tools')
sys.path.append('/users/taylordupuy/Documents/web-development/dev/email_tools')

from table_editor import SheetObject

R=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster.xlsx','roster')
S=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster.xlsx','submissions')
A=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster.xlsx','assignments')
P=SheetObject('/users/taylordupuy/Documents/web-development/data/algebra-one/20/f/roster-test.xlsx','assignments')

In [5]:
#doesn't work
#os.remove('db.sqlite')

In [3]:
from database_declarative import User, Submission, Problem, Course, Base
#from models import db
#from models import User, Submission, Problem, Course
from sqlalchemy import create_engine
import numpy as np
from email_functions import *
from sqlalchemy import or_, and_

In [4]:
engine = create_engine('sqlite:///db.sqlite')
Base.metadata.bind = engine
from sqlalchemy.orm import sessionmaker
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

In [6]:
#from main import sort_subs_by_stage

In [5]:
def is_reviewed(s):
    return s.review1_timestamp>-1 and s.review2_timestamp>-1

def is_complete(s):
    if is_reviewed(s):
        a=s.assignment
        p=s.problem
        r1 = s.reviewer1
        r2 = s.reviewer2
        subs = session.query(Submission).filter( and_(or_(Submission.netid==r1,Submission.netid==r2),Submission.assignment ==a,Submission.problem ==p))
        if is_reviewed(subs[0]) and is_reviewed(subs[1]):
            return True
        else: 
            return False
    else:
        return False
    
def is_matched(s):
    if s.reviewer1_assignment_time>-1 and s.reviewer2_assignment_time>-1:
        return True
    else:
        return False
    
def get_grade(user,ass):
    """
    get_grade('yshi9',1)
    7.875473484848484
    """
    subs=session.query(Submission).filter( and_(Submission.netid==user, Submission.assignment==ass))
    subs = sorted(subs, key=lambda s: s.total_score2)
    return np.mean([s.total_score2 for s in subs[0:2]])

def get_email(netid):
    user=session.query(User).filter(User.netid==netid).first()
    return user.email

In [6]:
# Insert an Address in the address table
algebra = Course(
    coursename = 'algebra-one',
    year=20,
    term = 'f', #F=Fall, S=Spring, U=Summer,
    coursenumber = 'Math 251', #used to be class
    uvm_course = 1,
    zulipurl = 'https://uvm-mathematics.zulipchat.com',
    zuliprc = '',
    homepage = 'https://www.uvm.edu/~tdupuy/',
    size = 0
)
session.add(algebra)
session.commit()

In [7]:
# Insert an Address in the address table
algebraic_topology = Course(
    coursename = 'algebraic-topology',
    year=20,
    term = 'f', #F=Fall, S=Spring, U=Summer,
    coursenumber = 'Math 354', #used to be class
    uvm_course = 1,
    zulipurl = 'https://uvm-mathematics.zulipchat.com',
    zuliprc = '',
    homepage = 'https://www.uvm.edu/~tdupuy/',
    size = 0
)
session.add(algebraic_topology)
session.commit()

In [8]:
session.query(Course).all()[0].term
#session.delete(session.query(Course).first())

'f'

In [10]:
"""
from sqlalchemy.ext.declarative import DeclarativeMeta

class AlchemyEncoder(json.JSONEncoder):

    def default(self, obj):
        if isinstance(obj.__class__, DeclarativeMeta):
            # an SQLAlchemy class
            fields = {}
            for field in [x for x in dir(obj) if not x.startswith('_') and x != 'metadata']:
                data = obj.__getattribute__(field)
                try:
                    json.dumps(data) # this will fail on non-encodable values, like other classes
                    fields[field] = data
                except TypeError:
                    fields[field] = None
            # a json-encodable dict
            return fields

        return json.JSONEncoder.default(self, obj)
    """

"\nfrom sqlalchemy.ext.declarative import DeclarativeMeta\n\nclass AlchemyEncoder(json.JSONEncoder):\n\n    def default(self, obj):\n        if isinstance(obj.__class__, DeclarativeMeta):\n            # an SQLAlchemy class\n            fields = {}\n            for field in [x for x in dir(obj) if not x.startswith('_') and x != 'metadata']:\n                data = obj.__getattribute__(field)\n                try:\n                    json.dumps(data) # this will fail on non-encodable values, like other classes\n                    fields[field] = data\n                except TypeError:\n                    fields[field] = None\n            # a json-encodable dict\n            return fields\n\n        return json.JSONEncoder.default(self, obj)\n    "

In [11]:
session.query(User).all()

In [9]:
myusers = R.get({})

# Insert a Person in the person table
for x in myusers:
    new_person = User(
        name = x['name'],
        pronoun=x['pronoun'],
        uvm_student=1,
        student_id = x['student_id'],
        courses = 'algebra-one/f/20',
        major = x['major'],
        degree = x['degree'],
        credits = x['credits'],
        reg_status = x['reg_status'],
        reg_data = x['reg_data'],
        email = x['email'].lower(),
        netid = x['netid'],
        password = generate_password_hash(x['password'],method='sha256')
    )
    session.add(new_person)

session.commit()

In [13]:
#session.delete(session.query(User).first())
[user.netid for user in session.query(User).all()]

['ddeckers',
 'yhao3',
 'chliberm',
 'tdupuy',
 'swan',
 'ttamang',
 'eepearso',
 'mkbeck',
 'agilland',
 'rabbott1',
 'gwonnell',
 'sdavis38',
 'rgoldma1',
 'acreigh2',
 'hvercess',
 'tfbloom',
 'jwang49',
 'rtdownin',
 'asilence',
 'cdecours',
 'keveronn',
 'phunter1',
 'nhanoian',
 'whuang3',
 'cbremner',
 'hgal',
 'yohno',
 'qsheng1',
 'she1',
 'moconno7',
 'hpeng1',
 'whe3',
 'hmagee',
 'yshi9',
 'rsantell',
 'dwoods2',
 'ddeckers',
 'yhao3',
 'chliberm',
 'tdupuy',
 'swan',
 'ttamang',
 'eepearso',
 'mkbeck',
 'agilland',
 'rabbott1',
 'gwonnell',
 'sdavis38',
 'rgoldma1',
 'acreigh2',
 'hvercess',
 'tfbloom',
 'jwang49',
 'rtdownin',
 'asilence',
 'cdecours',
 'keveronn',
 'phunter1',
 'nhanoian',
 'whuang3',
 'cbremner',
 'hgal',
 'yohno',
 'qsheng1',
 'she1',
 'moconno7',
 'hpeng1',
 'whe3',
 'hmagee',
 'yshi9',
 'rsantell',
 'dwoods2']

In [10]:
myproblems = A.get({})

for x in myproblems:
    new_problem = Problem(
    coursename = 'algebra-one', #algebra-one, algebraic-topology, agittoc1, agittoc2, agittoc3
    year = 2020,
    term = 'f', #f=Fall, s=Spring, u=Summer
    assignment = x['assignment'],
    problem = x['problem'],
    #references = x['']
    )
    session.add(new_problem)
    session.commit()

In [11]:
mysubs = S.get({})
for x in mysubs:
    new_sub = Submission(
        coursename = 'algebra-one',
        year = '2020',
        term = 'f',
        submission_number = x['submission_number'],
        submission_locked = x['submission_locked'],
        netid = x['netid'],
        email = get_email(x['netid']),
        assignment = x['assignment'],
        problem = x['problem'],
        closed = x['closed'],
        submission_time = x['submission_time'],
        total_score1 = x['total_score1'],
        total_score2 = x['total_score2'],
        reviewer1_assignment_time =x['reviewer1_assignment_time'],
        reviewer1 = x['reviewer1'],
        reviewer1_email = '',
        reviewer1_score = x['reviewer1_score'],
        review1 = x['review1'],
        review1_timestamp = x['review1_timestamp'],
        review1_locked = x['review1_locked'],
        #bad1, bad2
        reviewer2_assignment_time =x['reviewer2_assignment_time'],
        reviewer2 = x['reviewer2'],
        reviewer2_email = '',
        reviewer2_score = x['reviewer2_score'],
        review2 = x['review2'],
        review2_timestamp = x['review2_timestamp'],
        review2_locked = x['review2_locked'],
        new_submission = x['new_submission'],
        new_match = x['new_match'],
        new_review1 = x['new_review1'],
        new_review2 = x['new_review2'],
        new_completion = x['new_completion']
        )
    session.add(new_sub)

session.commit()

In [14]:
#need to correct the submissions
for s in session.query(Submission).all():
    if is_matched(s):
        s.reviewer1_email=get_email(s.reviewer1)
        s.reviewer2_email=get_email(s.reviewer2)

In [15]:
#this needs to be fixed for AGITTOC
student = 'tfbloom'
myuser = session.query(User).filter(User.netid==student).first()
email=myuser.email
subs=session.query(Submission).filter(Submission.email==email)
email

'tyler.bloom@uvm.edu'

In [29]:
reviewsubs = session.query(Submission).filter( or_(Submission.reviewer1_email==email,Submission.reviewer2_email==email)).all()
reviewsubs

In [20]:
fresh_subs=session.query(Submission).filter(and_(Submission.netid==student, Submission.new_submission==1) ).all()
matched_subs=session.query(Submission).filter(and_(Submission.netid==student, Submission.submission_locked==1)).all()

first_reviews = []
for s in matched_subs:
    if is_reviewed(s):
        first_reviews.append(s)

for c in first_reviews:
    matched_subs.remove(c)        
        
completed =[]
for s in first_reviews:
    if is_complete(s):
        completed.append(s)
        
for c in completed:
    first_reviews.remove(c)

fresh_subs=sorted(fresh_subs,key=lambda c: [c.assignment,c.problem])
matched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])
first_reviews = sorted(first_reviews,key=lambda c: [c.assignment,c.problem])
completed=sorted(completed,key=lambda c: [c.assignment,c.problem])


In [19]:
for s in fresh_subs:
    print(f"{s.submission_number} (waiting): assignment {s.assignment}, problem {s.problem}")
    
for s in matched_subs:
    print(f"{s.submission_number} (matched): assignment {s.assignment}, problem {s.problem}")
    
for s in first_reviews:
    print(f"{s.submission_number} (first review done): assignment {s.assignment}, problem {s.problem}")
    
for s in completed:
    print(f"{s.submission_number} (complete): assignment {s.assignment}, problem {s.problem}")
    

NameError: name 'fresh_subs' is not defined

In [12]:
"""
completed =[]
for s in subs:
    if is_complete(s):
        completed.append(s)

fresh_subs=sorted(fresh_subs,key=lambda c: [c.assignment,c.problem])
matched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])
completed=sorted(completed,key=lambda c: [c.assignment,c.problem])
"""

'\ncompleted =[]\nfor s in subs:\n    if is_complete(s):\n        completed.append(s)\n\nfresh_subs=sorted(fresh_subs,key=lambda c: [c.assignment,c.problem])\nmatched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])\ncompleted=sorted(completed,key=lambda c: [c.assignment,c.problem])\n'

In [13]:
for c in completed:
    print(f"{c.submission_number}: assignment {c.assignment}, problem {c.problem}, score {c.total_score2}")

In [14]:
closed_subs=[s.submission_number for s in subs.filter(Submission.closed==1)]

completed

In [15]:
s=subs[0]

IndexError: list index out of range

In [35]:
s.reviewer1
s.reviewer2

'cdecours'

In [36]:
is_reviewed(s)
a=s.assignment
p=s.problem
r1 = s.reviewer1
r2 = s.reviewer2
print(f'submission number: {s.submission_number}, author: {s.netid}, reviewer 1: {r1}, reviewer2: {r2}') 
reviews = session.query(Submission).filter( and_(or_(Submission.netid==r1,Submission.netid==r2),Submission.assignment ==a,Submission.problem ==p))
for rev in reviews:
    print(f" submission number: {rev.submission_number}; assignment {rev.assignment}, problem {rev.problem}, reviewer id {rev.netid}")

submission number: 92, author: yshi9, reviewer 1: ttamang, reviewer2: cdecours
 submission number: 70; assignment 1, problem 1, reviewer id cdecours
 submission number: 79; assignment 1, problem 1, reviewer id ttamang


In [45]:
is_complete(s)

False

In [ ]:
s.review1_timestamp

In [37]:
def poke_reviewers(sub_num):
    subject = "re"
    sub=session.query(Submission).filter(Submission.submission_number=sub_num)
    if s.review1_timestamp==-1:
        myemail['sender'] == 
        out_msg=send_email(myemail)
    if s.review2_timestamp==-1:        

SyntaxError: invalid syntax (<ipython-input-37-db720c296e9f>, line 5)

In [ ]:
    emails[user]['subject'] = course_name + " Database Updates " + ("%s" % datetime.datetime.now())
    #emails[user]['message_parts'] = {}
    emails[user]['message'] =''
    emails[user]['attachments'] = []
    emails[user]['receiver'] = R.get({'netid':user})[0]['email']

In [17]:
matched_subs = session.query(Submission).filter( or_(Submission.netid==student,Submission.netid==student)).all()

In [18]:
first_reviews = []
for s in matched_subs:
    if is_reviewed(s):
        first_reviews.append(s)

for c in first_reviews:
    matched_subs.remove(c)        
        
completed =[]
for s in first_reviews:
    if is_complete(s):
        completed.append(s)
        
for c in completed:
    first_reviews.remove(c)

matched_subs=sorted(matched_subs,key=lambda c: [c.assignment,c.problem])
first_reviews = sorted(first_reviews,key=lambda c: [c.assignment,c.problem])
completed=sorted(completed,key=lambda c: [c.assignment,c.problem])

In [16]:
session.query(Course).all()[0].coursename

'algebra-one'